In [ ]:
# Import necessary libraries
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# We do this to add the correct module to our python path.
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.Simulation import run_simulation
from src.BuildSimulation import BuildSimulation
from src.util import get_risk_prob_proportions

In [ ]:
# Declaring some variables that are used in the simulations.
filename = "results"
setup = BuildSimulation()
risks = np.linspace(0.0, 1.0, 21)  # 0.05 step size, open intervals

In [ ]:
# Running the simulations. 
# UNCOMMENT THE FOLLOWING IF YOU WANT TO EXECUTE THE EXPERIMENTS.
'''
for risk in risks:
    print("Simulation for risk " + str(risk))
    setup.risk = risk
    savefile = filename + "-" + str(risk) + ".csv"
    run_simulation(setup, savefile)
'''

In [ ]:
# Loading the results from the files.
filename = "results"
filenames = [filename + "-" + str(risk) + ".csv" for risk in risks]
filenames = list(map(lambda f: os.path.join(module_path,f),filenames))
results = []
for filename in filenames:
    result = pd.read_csv(filename)
    # Make sure the list has the right format, otherwise we cannot perform operations on it.
    for i in range(setup.num_generations):
        tmp = ast.literal_eval(result["rounds_contributions_counts"][i])
        result["rounds_contributions_counts"][i] = tmp
        result["behaviors_counts"][i] = ast.literal_eval(result["behaviors_counts"][i])
    results.append(result)

    
#for result in results:
#    print(result)

In [ ]:
# Getting all variables that will be used in plotting future figures
payoffs, targets, contribs, fh_contribs, sh_contribs = get_risk_prob_proportions(setup, risks, results)
generations = range(1, setup.num_generations + 1)

In [ ]:
# Plot the risk probability proportions
plt.figure()
plt.plot(risks, payoffs, label="Payoff")
plt.plot(risks, contribs, label="Contribution")
plt.plot(risks, targets, label="Target")
plt.plot(risks, fh_contribs, label="First half contributions")
plt.plot(risks, sh_contribs, label="Second half contributions")
plt.ylim((0, 1))
plt.xlabel("Risk probability")
plt.ylabel("Proportion")
plt.legend()
plt.show()

In [ ]:
# Plot the trajectories for all the results.
for result in results: 
    # Get the values used in the plot.
    payoffs = result["avg_payoffs"]
    targets = result["targets_reached"]
    rounds_contributions_counts = np.array(result["rounds_contributions_counts"].tolist())

    rounds_contributions = np.sum(rounds_contributions_counts * setup.legal_moves, axis=2) / np.sum(rounds_contributions_counts, axis=2)
    contributions = np.sum(rounds_contributions, axis=1)
    
    # Plot the actual figure.
    plt.figure(figsize=(16, 4))
    plt.plot(generations, payoffs / setup.initial_endowment, label="Payoff")
    plt.plot(generations, contributions / setup.initial_endowment, label="Contribution")
    plt.plot(generations, targets / setup.num_games, label="Target")
    plt.ylim((0, 1))
    plt.xlabel("Generations")
    plt.ylabel("Proportion")
    plt.legend()
    plt.show()